In [323]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [324]:
import seaborn as sns
import matplotlib as plt

In [325]:
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [326]:
sample = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

## Exploration and Visualizations

In [327]:
train.describe()

In [328]:
sns.heatmap(train.isnull(), cbar=False)

In [329]:
train.drop(columns = ['Alley','FireplaceQu'], inplace = True)
test.drop(columns = ['Alley','FireplaceQu'], inplace = True)

In [330]:
sns.heatmap(train.isnull(), cbar=False)

In [331]:
train.columns

In [332]:
train.corr()

In [333]:
sns.heatmap(train.corr());

In [334]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(train[cols], height = 2.5)
plt.pyplot.show();

In [335]:
sns.histplot(data = train, x = 'SalePrice')

In [336]:
sns.scatterplot(data = train, x = 'LotArea', y = 'SalePrice')

In [337]:
len(train.columns)

In [338]:
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [339]:
train = train.drop((missing_data[missing_data['Total'] > 1]).index,1)
train = train.drop(train.loc[train['Electrical'].isnull()].index)
train.isnull().sum().max()

In [340]:
train

In [341]:
test = test.drop((missing_data[missing_data['Total'] >= 5]).index,1)
test = test.drop(train.loc[train['Electrical'].isnull()].index)
test.isnull().sum().max()

In [342]:
test

In [343]:
for column in test.columns:
    mode = test[column].mode()
    test[column].fillna(value = mode, inplace = True)

In [344]:
test

In [345]:
#sns.distplot(train['SalePrice'], fit=norm);
#fig = plt.figure()
#res = stats.probplot(train['SalePrice'], plot=plt)

In [346]:
#sns.distplot(train['SalePrice'], fit=norm);
#fig = plt.figure()
#res = stats.probplot(train['SalePrice'], plot=plt)

In [347]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [348]:
test

In [349]:
test.isnull().sum().max()

In [350]:
test.fillna(0, inplace = True)

In [351]:
SalePrice = train['SalePrice']

In [352]:
train = train[test.columns]

## Regression

In [353]:
from sklearn.linear_model import LinearRegression

In [354]:
reg = LinearRegression()

In [356]:
labels = SalePrice
conv_dates = [1 if values == 2014 else 0 for values in train.YrSold ]
train['date'] = conv_dates
train1 = train.drop(['Id'],axis=1)

In [357]:
from sklearn.model_selection import train_test_split

In [358]:
x_train , x_test , y_train , y_test = train_test_split(train1 , labels , test_size = 0.01,random_state =969)

In [359]:
reg.fit(x_train,y_train)

In [360]:
reg.score(x_test,y_test)

In [361]:
from sklearn import ensemble
clf = ensemble.GradientBoostingRegressor(n_estimators = 400, max_depth = 5, min_samples_split = 2,
          learning_rate = 0.1, loss = 'ls')

In [362]:
clf.fit(x_train, y_train)

In [363]:
clf.score(x_test,y_test)

In [364]:
clf.score

In [366]:
test_predict = clf.predict(test)

In [367]:
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': test_predict})

In [369]:
my_submission.to_csv('submission.csv', index=False)